In [2]:
import pandas as pd 
import numpy as np 
from sklearn import preprocessing 
from sklearn import utils 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

This code is submitted as part of project 2 for the subject COMP90037 (Security Analytics) at the University of  Melbourne .
     
    -------------------------------------------
    COMP90037 Security Analytics - Project 2 
    Machine learning based Threat detection

    Author : Mohammed Ahsan Kollathodi 
    Student id: 1048942.
    

### The Logistic Regression Classifier 

The logistic regression is one of the most common classification algorithm that's widely used in machine learning problems. It is used when the value of the targert variable is categorical in nature. Logistic regression is most commonly used when the data that we could consider has a binary output, so it belongs to one class or another with values of 0 or 1. 

Here by the idea of regression model to solve the classification problem, we would rationally raise a question of whether we can draw a hypothesis function to fit to the binary dataset. 

Reference : https://kambria.io/blog/logistic-regression-for-machine-learning/



### Model

In [3]:
train = pd.read_csv('training_data_with_ip_A2.csv', sep=',')

In [4]:
# Train set of features. 
X_train = train.drop(['Botnet','src_ip'],1)

# Train set of labels. 
Y_train = train[['Botnet']]

# Encode the labels using label Encoder from sklearn. 
lab_enc = preprocessing.LabelEncoder() 


# Encode the train set labels with the label encoder. 
trainencoded = lab_enc.fit_transform(Y_train)
print(trainencoded)



[  0   0 418 ...   0   0   0]


/var/folders/3c/0p84rf_94y92bbvwmt4vtjl00000gn/T/ipykernel_96191/3253761529.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train = train.drop(['Botnet','src_ip'],1)
/Users/ahsan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [6]:
# reshape the set of labels using ravel. 
Y_train = np.ravel(trainencoded)
print(Y_train)

# Build the LR model . 
model = LogisticRegression(solver = 'lbfgs', max_iter=1000).fit(X_train, Y_train.astype(int))

[  0   0 418 ...   0   0   0]


/Users/ahsan/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
test = pd.read_csv('test_data_with_ip_A2.csv', sep=',')

Train and Test data set format : 
-------------------------------------------------

        Column no.                        name 
                    
                1.                        src_ip
                2.                        duration 
                3.                        num_total_packets
                4.                        total_bytes
                5.                        src_bytes
                6.                        packets_in_Sec
                7.                        bytes_total_in_Sec
                8.                        Source_bytes_Sec
                9.                        Botnet - label    
            
           
            




In [ ]:
X_test = test.drop(['Botnet','src_ip'],1)
Y_test = test[['Botnet']]

In [ ]:
# Encode the test set labels with the label encoder. 
testencoded = lab_enc.fit_transform(Y_test)

# reshape the test labels using ravel. 
Y_test = np.ravel(Y_test)


In [ ]:
X_test.head()

In [10]:
# Generate the model score with the test set features and label encoded test labels. 
# testencoded is the Y_test encoded with labels. 
model.score(X_test,testencoded)  

0.8611876250955625

In [11]:
# predicted set of values. 
predict_value = pd.DataFrame(model.predict(X_test),columns=['Predictions'])

In [16]:
# Final result with the source ip and Botnet. 
final_result = pd.concat([test[['src_ip','Botnet']].copy(),predict_value], axis=1)

In [17]:
# display the result 
final_result.head(5)


,src_ip,Botnet,Predictions
0,100.100.101.69,0.000000,0
1,100.100.117.223,0.000000,0
2,100.100.140.59,0.000000,0
3,100.100.151.53,0.000000,0
4,100.100.163.162,0.333333,0


### Adversarial sample 

What is an adversarial sample ? 

An adversarial sample is an instance with small, intentional feature perturbations that would cause a machine learning model to result in false predictions. 

[Reference : https://towardsdatascience.com/adversarial-examples-in-deep-learning-be0b08a94953]



In [18]:
ip_addr = X_test.iloc[[1]]

In [19]:
ytest= test[['Botnet']]

In [20]:
label =  ytest.iloc[[1]]

In [21]:
# Adverserial sample. 

epsilon = 50
direction = 1 

# Adversarial sample - adv_sample . 
adv_sample = ip_addr.values + epsilon * direction

In [22]:
print(adv_sample)

[[1762.70057078   54.5        1150.          998.           68.55586618
  9568.8065401  8455.48901833]]


In [23]:
X_test.iloc[[1]] = adv_sample

In [24]:
X_test.head()

,duration,num_total_packets,total_bytes,src_bytes,packets_in_Sec,bytes_total_in_Sec,Source_Bytes_Sec
0,0.001515,2.000000,127.0,59.000000,1320.469628,83849.821403,38953.854037
1,1762.700571,54.500000,1150.0,998.000000,68.555866,9568.806540,8455.489018
2,0.000563,2.000000,127.0,60.000000,3553.666588,225657.828334,106609.997638
3,1169.036049,3.333333,307.0,169.666667,879.848846,145174.861898,92383.984864
4,1017.529307,4.333333,3087.0,120.000000,985.382515,235806.245078,75458.505848


In [25]:
predictions = pd.DataFrame(model.predict(X_test),columns=['Predictions'])

In [26]:
final_result = pd.concat([test[['src_ip','Botnet']].copy(),predictions], axis=1)

In [27]:
model.score(X_test,testencoded)

0.8611876250955625

In [28]:
final_result.head()

,src_ip,Botnet,Predictions
0,100.100.101.69,0.000000,0
1,100.100.117.223,0.000000,0
2,100.100.140.59,0.000000,0
3,100.100.151.53,0.000000,0
4,100.100.163.162,0.333333,0


### Model with perturbed data 

In [29]:
perturbed_test = pd.read_csv('test_data_with_ip_perturbed_A2.csv', sep=',')

In [30]:
# perturbed test set features. 

X_test_perturbed_set = perturbed_test.drop(['Botnet','src_ip'],1)

# perturbed test set labels.

Y_test_perturbed_set = perturbed_test[['Botnet']]

/var/folders/3c/0p84rf_94y92bbvwmt4vtjl00000gn/T/ipykernel_30851/2439567288.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_test_perturbed_set = perturbed_test.drop(['Botnet','src_ip'],1)


In [35]:
perturbed_test.head()

# Encode the test labels. 

testpert_encoded = lab_enc.fit_transform(Y_test_perturbed_set)

/Users/ahsan/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [36]:
# model score for the perturbed test dataset with ip. 
model.score(X_test_perturbed_set,testpert_encoded)

0.8611876250955625

In [37]:
# predictions. 
predictions = pd.DataFrame(model.predict(X_test_perturbed_set),columns=['Predictions'])

In [38]:
from sklearn.metrics import confusion_matrix, accuracy_score
# Accuracy and confusion matrix. 
# accuracy = accuracy_score(y_test, y_pred)
accuracy = accuracy_score(testpert_encoded,predictions)
print("accuracy :")
print(accuracy)
# Confusion matrix. 
# Confusion matrix = confusion_matrix(y_test, y_pred)
print("Confusion matrix:")
confusion_matrix = confusion_matrix(testpert_encoded,predictions)
print(confusion_matrix)


accuracy :
0.8611876250955625
Confusion matrix:
[[224168      4      0      0      0      0      0      0]
 [  2088      0      0      0      0      0      0      0]
 [     1      0      0      0      0      0      0      0]
 [     1      0      0      0      0      0      0      0]
 [     1      0      0      0      0      0      0      0]
 [     1      0      0      0      0      0      0      0]
 [ 12186      0      0      0      0      0      0      0]
 [ 21850      1      0      0      0      0      0      0]]


In [39]:
# Final result. 
final_result = pd.concat([perturbed_test[['src_ip','Botnet']].copy(),predictions], axis=1)

In [1]:
final_result.head(4)

NameError: name 'final_result' is not defined